## Install Important Dependencies (Libraries)

In [ ]:
!pip install wandb evaluate rouge_score datasets wikipedia-api

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5d1c88469b1a4b26c788aef59cab02820ce19a2c57e0dc7358c78a4d28b7f5dc
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=944e3b5f1a394b096f99859ae88bb3288c3bf2e56e092cae280edbfc21e84f77
  Sto




##Aquire articles




In [ ]:
import requests
import time


TARGET_ARTICLES = 500 # Number of articles to fetch
BATCH_SIZE = 50  # Number of articles per API call

def get_articles(target_count = TARGET_ARTICLES, batch_size = BATCH_SIZE ):

    """
    Fetches exactly random Wikipedia article titles, filtering out non-article pages.
    Returns list of a specified number of articles
    """

    url = "https://en.wikipedia.org/w/api.php"
    filter_keywords = ["File talk:", "Category talk:", "Portal talk:", "User:", "Talk:", "Category:",
                       "Template:", "Wikipedia:", "User talk:", "File:", "Template talk:" , "Draft:",
                       "Time talk:", "Wikipedia talk:"] #filter to get articles

    valid_articles = set() #avoiding Duplicates

    while len(valid_articles) < target_count:
        params = {
            "action": "query",
            "format": "json",
            "list": "random",
            "rnlimit": batch_size #optimization using batch
        }

        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}. Trying again !")
            time.sleep(2)
            continue

        try:
            data = response.json()
            articles = [
                item['title'] for item in data['query']['random']
                if not any(prefix in item['title'] for prefix in filter_keywords)
            ]

            valid_articles.update(articles)  # Add only valid articles to the set

        except Exception as e:
            print(f"Error processing response: {e}")
            time.sleep(2)

    # Convert to list and trim to exactly 500 articles
    valid_articles = list(valid_articles)[:target_count]

    return valid_articles

topics = get_articles()


# Print first 10 valid Wikipedia articles
for i in range(10):
    print(topics[i])

print("-------------------------------------------------------------------------------")

# Print the total number of valid Wikipedia articles
print(f"Total valid Wikipedia articles collected: {len(topics)}")

Grammy Award for Best Remixed Recording, Non-Classical
The Tamale Vendor
Donald Von Raesfeld Power Plant
List of mountains of Vaud
Gustav Leonhardt
La Bretagne ouvrière, paysanne et maritime
Periaptodes testator
Santi Bartolomeo ed Alessandro dei Bergamaschi
1980 Emilia-Romagna regional election
Flat Island Wildlife Sanctuary
-------------------------------------------------------------------------------
Total valid Wikipedia articles collected: 500


## Acquire Text for articles

In [ ]:
import wikipediaapi
import json
import time

USER_AGENT = "MyWikipediaScraper/1.0 (contact: krishnanand.dma@gmail.com)"

# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(user_agent=USER_AGENT, language="en")

def get_articles(topics):

    """
    Fetches Wikipedia content for a given list of topics.
    """
    articles = {}

    for i, title in enumerate(topics):
        page = wiki_wiki.page(title)

        if page.exists():
            articles[title] = page.text
            print(f"[{i+1}/{len(topics)}] Fetched: {title}")
        else:
            print(f"[{i+1}/{len(topics)}] Skipped: {title} (Page does not exist)")

        # API limits to avoid getting blocked
        time.sleep(0.5)

    return articles

# Fetch Wikipedia articles calling  get_articles function
wikipedia_data = get_articles(topics)

# Save data as JSON
output_file = "wikipedia_articles.json"
with open(output_file, "w") as f:
    json.dump(wikipedia_data, f, indent=4)

print(f"Successfully saved Wikipedia articles to {output_file}")

[1/500] Fetched: Grammy Award for Best Remixed Recording, Non-Classical
[2/500] Fetched: The Tamale Vendor
[3/500] Fetched: Donald Von Raesfeld Power Plant
[4/500] Fetched: List of mountains of Vaud
[5/500] Fetched: Gustav Leonhardt
[6/500] Fetched: La Bretagne ouvrière, paysanne et maritime
[7/500] Fetched: Periaptodes testator
[8/500] Fetched: Santi Bartolomeo ed Alessandro dei Bergamaschi
[9/500] Fetched: 1980 Emilia-Romagna regional election
[10/500] Fetched: Flat Island Wildlife Sanctuary
[11/500] Fetched: Göttin
[12/500] Fetched: Guillermina Dulché
[13/500] Fetched: Waltraut Cooper
[14/500] Fetched: Anthela callispila
[15/500] Fetched: Adrienne Johnson (basketball)
[16/500] Fetched: 2020 in the United Nations
[17/500] Fetched: Abasi
[18/500] Fetched: Three Way
[19/500] Fetched: Prunus 'Kanzan'
[20/500] Fetched: Claudia de Breij
[21/500] Fetched: Melissa Brown (tennis)
[22/500] Fetched: Ethmia zebrata
[23/500] Fetched: Edward L. Hearn
[24/500] Fetched: Syngamus
[25/500] Fetched: P

## Normalization of Acquired Text

In [ ]:
import json
import unicodedata

# Normalize to standard ASCII on the articles data.
def normalizeDataset(text):
    return unicodedata.normalize('NFKC', text) if isinstance(text, str) else text

input_file = "wikipedia_articles.json"
output_file = "wikipedia_articles_cleaned.json"

with open(input_file, "r", encoding="utf-8") as f:
    wikipedia_data = json.load(f)

# Apply normalization
cleaned_data = {title: normalizeDataset(content) for title, content in wikipedia_data.items()}

# Save
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=4, ensure_ascii=False)

print(f"Cleaned dataset saved to {output_file}")

Cleaned dataset saved to wikipedia_articles_cleaned.json


## Generating Q-A pairs: Required for Training

In [ ]:
import json
import torch
from transformers import pipeline


input_file_path = "wikipedia_articles_cleaned.json"
output_file_path = "qa_pairs_cleaned.json"

with open(input_file_path, "r", encoding="utf-8") as file:
    wikipedia_articles = json.load(file)

num_qa = 500
selected_articles = list(wikipedia_articles.items())[:num_qa]

# Load text generation model
qa_pair_generator = pipeline("text-generation", model="mosaicml/mpt-7b-instruct", device=0, torch_dtype=torch.float16)

# empty list to store Q&A pairs
qa_pairs = []

# Process each article
for title, content in selected_articles:

    chunk = content[:1500] #extracting chunks

    # the prompt to fed in the model
    prompt = f"""
    Strictly generate structured question-answer pairs from Wikipedia articles.
    Generate exactly {10} question-answer pairs in this format:

    Q: What is Artificial Intelligence?
    A: Artificial Intelligence is the simulation of human intelligence in machines.

    Q: Who is Andreas Koukouma?
    A: Andreas Koukouma is a Cypriot politician and poet.

    Q: Where is the Eiffel Tower located?
    A: The Eiffel Tower is located in Paris, France.

    Q: Is the Earth flat?
    A: No, the Earth is not flat.

    Now generate Q&A pairs for the following article:

    Title: {title}
    Content:
    {chunk}

    Start the Q&A list below:
    """

    # generate Q&A pairs
    qa_output = qa_pair_generator(prompt, max_length= 1024, do_sample=True, temperature=0.7)

    generated_text = qa_output[0]["generated_text"]
    cleaned_qa_pairs = generated_text.split("Start the Q&A list below:")[-1].strip()

    #for debugging
    print(f"Generated Q&A for: {title}")
    print(cleaned_qa_pairs)
    print("-" * 80)  # Separator

    # Append cleaned Q&A pairs with title
    qa_pairs.append({"title": title, "qa_pairs": cleaned_qa_pairs})

    # write each generated pair directly to JSON
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        json.dump(qa_pairs, outfile, indent=4, ensure_ascii=False)

print(f"QA pairs saved to {output_file_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

...

Generated Q&A for: Gurudhakshaneiy
Q: What is Artificial Intelligence?
    A: Artificial Intelligence is the simulation of human intelligence in machines.

    Q: Who is Andreas Koukouma?
    A: Andreas Koukouma is a Cypriot politician and poet.

    Q: Where is the Eiffel Tower located?
    A: The Eiffel Tower is located in Paris, France.

    Q: Is the Earth flat?
    A: No, the Earth is not flat.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Generated Q&A for: Paweensuda Drouin
Q: What is the Eiffel Tower located in?
    A: Paris, France.
    Q: Who is Andreas Koukouma?
    A: Andreas Koukouma is a Cypriot politician and poet.
    Q: What is the capital of Thailand?
    A: Bangkok.
    Q: When was the Eiffel Tower built?
    A: The Eiffel Tower was built in 1889.
    Q: When did Jennifer Paweensuda Drouin win Miss Universe Thailand?
    A: Jennifer Paweensuda Drouin won Miss Universe Thailand in 2019.
    Q: When did Jennifer Paweensuda Drouin win Miss Thailand 2013?
    A: Jennifer Paweensuda Drouin won Miss Thailand 2013 in 2013.
    Q: How old was Jennifer Paweensuda Drouin when she won Miss Thailand 2013?
    A: Jennifer Paweensuda Drouin was 20 years old when she won Miss Thailand 2013.
    Q: Who is Jennifer Paweensuda Drouin's mother?
    A: Jennifer Paweensuda Drouin's mother is Thai Chinese.
    Q: Who is Jennifer Paweensuda Drouin's father?
    A: Jennifer Paweensuda Drouin's father is of Cuban-French descent.
  

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Generated Q&A for: The Well of Loneliness
Q: What is the Well of Loneliness about?
    A: The Well of Loneliness is a lesbian novel by British author Radclyffe Hall that was first published in 1928 by Jonathan Cape. It follows the life of Stephen Gordon, an Englishwoman from an upper-class family whose "sexual inversion" (homosexuality) is apparent from an early age. She finds love with Mary Llewellyn, whom she meets while serving as an ambulance driver during the First World War, but their happiness together is marred by social isolation and rejection, which Hall depicts as the typical sufferings of "inverts", with predictably debilitating effects. The novel portrays "inversion" as a natural, God-given state and makes an explicit plea: "Give us also the right to our existence".
    Shortly after the book's publication, it became the target of a campaign by James Douglas, editor of the Sunday Express. Douglas wrote that "I would rather give a healthy boy or a healthy girl a phial of pr

## Cleaning Q-A pairs: Removal of Unwanted Information

In [ ]:
import json
import re

input_file = "qa_pairs_cleaned.json"
output_file = "formatted_qa_pairs.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

cleaned_qa_list = []

for entry in data:
    qa_text = entry["qa_pairs"]

    # Regex pattern to extract Q/A pairs
    qa_pairs = re.findall(r'\s*Q:\s*(.*?)\n\s*A:\s*(.*?)(?=\n\s*Q:|\Z)', qa_text, re.DOTALL)

    for question, answer in qa_pairs:
        cleaned_qa_list.append({
            "Question": question.strip(),
            "Answer": answer.strip()
        })

# Save the formatted Q:A pairs to json
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(cleaned_qa_list, f, indent=4)

print(f"Formatted Q:A pairs saved to {output_file}")

Formatted Q:A pairs saved to formatted_qa_pairs.json


## Converting Q-A pair to text

In [ ]:
import json

input_json = "formatted_qa_pairs.json"
output_txt = "QA_textform.txt"

#load
with open(input_json, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to text format required for training
with open(output_txt, "w", encoding="utf-8") as f:
    for item in data:
        question = item.get("Question", "").strip()
        answer = item.get("Answer", "").strip()

        if question and answer:
            if not answer.startswith("A:"): #answer
                answer = f"A: {answer}"

            f.write(f"Q: {question}\n{answer}\n\n") #questions

print(f"Converted Q&A data written to {output_txt}")

Converted Q&A data written to QA_textform.txt


## Converting Articles to Text for Training

In [ ]:
import json

input_file = "wikipedia_articles_cleaned.json"
output_file = "wikipedia_article_txtform.txt"

'''
converts articles to text format for training
'''

# inputs json and produces text

with open(input_file, "r", encoding="utf-8") as f:
    articles = json.load(f)

with open(output_file, "w", encoding="utf-8") as out:
    for article, content in articles.items():
        title = article.strip()
        text = content.strip()
        out.write(f"Title: {title}\n{text}\n\n")

## Combined and Shuffled Q-A Pair and Articles file

In [ ]:
import random

article_file = "wikipedia_article_txtform.txt"
qa_pair_file = "QA_textform.txt"
combined_file = "combined_train.txt"

''' Takes article file and q-a pair file as input returns shuffled and combined text file '''

# open both files and stripping the contents
with open(article_file, "r", encoding="utf-8") as f1, open(qa_pair_file, "r", encoding="utf-8") as f2:
    article_samples = f1.read().strip().split("\n\n")
    qa_samples = f2.read().strip().split("\n\n")

all_samples = article_samples + qa_samples # merging samples
random.shuffle(all_samples) # shuffling the contents

with open(combined_file, "w", encoding="utf-8") as out:
    out.write("\n\n".join(all_samples))

print("Shuffled and combined file saved as:", combined_file) # produce the combined file

Shuffled and combined file saved as: combined_train.txt


## Convert into train and validation set

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer


''' Tokenize the combined file and convert into train and validation sets '''

# Load GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 does not have a pad token

# prepare dataset
dataset = load_dataset("text", data_files={"train": "combined_train.txt"})

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split into training and validation sets
train_test_split = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/21693 [00:00<?, ? examples/s]

## Loading Model

In [ ]:
import torch
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Cleaning Cache
torch.cuda.empty_cache()

# Load GPT-2 small model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained("gpt2")


# Data collator for LM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

#for debugging
print("GPT-2 Model loaded successfully!")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT-2 Model loaded successfully!


## Defining Training Arguments

In [ ]:
import evaluate
import numpy as np
import os
import torch
from transformers import Trainer, TrainingArguments

# Set environment variable to avoid memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Free up GPU memory
torch.cuda.empty_cache()

# training arguments
training_args = TrainingArguments(
    output_dir="gpt2_finetuned_qa",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    bf16=True,
    dataloader_num_workers=4,
    report_to="none",  # Disable wandb ; otherwise requires key
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Training of Model ; saving the model and tokenizer


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("gpt2_finetuned_wikipedia")
tokenizer.save_pretrained("gpt2_finetuned_wikipedia_tok")

print("Fine-tuned GPT-2 model saved successfully!")

<ipython-input-7-8631c4c031cd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.114600,2.966774
2,2.307100,2.832033
3,1.714300,2.932728
4,1.007400,3.292023


Fine-tuned GPT-2 model saved successfully!


## Interacting with Model

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Choosing CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load fine-tuned model and tokenizer
model_name = "gpt2_finetuned_wikipedia"
tokenizer_name = "gpt2_finetuned_wikipedia_tok"

tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Set pad token
tokenizer.pad_token = tokenizer.eos_token

# Function for interacting with trained model
def ask_gpt2():
    print("\n Type your question (or type 'exit' to quit):")

    while True:
        question = input("📝 Ask a question: ").strip()
        if question.lower() == "exit":
            print("👋 Exiting the GPT-2 chatbot. Have a great day!")
            break

        # Format input in Q&A style
        input_text = f"Q: {question}\nA:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        # Generate response
        output = model.generate(
            input_ids,
            max_length=input_ids.shape[1] + 80,  # Control length
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,  # Use sampling for diverse responses
            top_p=0.9,  # Nucleus sampling
            temperature=0.7,  # Adjust randomness
            repetition_penalty=1.2,  # Avoid looping outputs
            early_stopping=True
        )

        # Decode response and remove input prompt
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        answer = generated_text.replace(input_text, "").strip()

        # Print answer
        print(f"\n🤖 Answer: {answer}\n")

# Run chatbot
ask_gpt2()


 Type your question (or type 'exit' to quit):
📝 Ask a question: who is einstein?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



🤖 Answer: He was a physics Nobel Prize winner in 2001. He was also the first person to have achieved this feat.  His next major achievement came in 2006 when he reached the limit of what he could achieve with an electron, which is described as reaching the limit of one quark at a time. He had succeeded Jan H Rabin in his previous attempts for large-scale quantum chromodynamics (QCD

📝 Ask a question: what is Artificial Intelligence ?

🤖 Answer: AI is the simulation of human intelligence in machines.  It is a field of study and development that aims to build computer programs capable of exhibiting intelligent behavior such as reasoning, learning, decision-making processes, natural language processing (A.I.)s., problem-solving systems, large datasets, etc.  However, researchers often have to rely on a technique known commonly referred to simply saying "

📝 Ask a question: exit
👋 Exiting the GPT-2 chatbot. Have a great day!
